Transactions are DB multiple updates that are to be done together. If any step in the update fail the whole transaction has to be rolled back. This has applications including in banking.
IN the example code below we are making a transaction where $100 is sent from sender to receiver. There are 2 collections called accounts and transfers. accounts document has details like id, balance and array of completed transfers.
This transaction will need 3 updates. All these updates need to be done by passing a client session object.
1. Decrease 100 from account balance of sender and list of completed transfers is updated with transaction ID
2. Increase 100 in account balance of receiver and list of completed transfers is updated with transaction ID
3. Update transaction details (like transaction ID) in transfers collection

In [ ]:
# Connect to MongoDB cluster with MongoClient
client = MongoClient(MONGODB_URI)

# Step 1: Define the callback that specifies the sequence of operations to perform inside the transactions.
# Here we are defining the update sequence
def callback(
    session, # this arguement is mandatory for a transaction. 
    transfer_id=None,
    account_id_receiver=None,
    account_id_sender=None,
    transfer_amount=None,
):

    # Get reference to 'accounts' collection
    accounts_collection = session.client.bank.accounts

    # Get reference to 'transfers' collection
    transfers_collection = session.client.bank.transfers

    # preparing a transfer document that needs to be added in the transfer collection
    transfer = {
        "transfer_id": transfer_id,
        "to_account": account_id_receiver,
        "from_account": account_id_sender,
        "amount": {"$numberDecimal": transfer_amount},
    }

    # Transaction operations
    # Important: You must pass the session to each operation

    # Update sender account: subtract transfer amount from balance and add transfer ID
    accounts_collection.update_one(
        {"account_id": account_id_sender},
        {
            "$inc": {"balance": -transfer_amount},
            "$push": {"transfers_complete": transfer_id},
        },
        session=session,
    )

    # Update receiver account: add transfer amount to balance and add transfer ID
    accounts_collection.update_one(
        {"account_id": account_id_receiver},
        {
            "$inc": {"balance": transfer_amount},
            "$push": {"transfers_complete": transfer_id},
        },
        session=session,
    )

    # Add new transfer to 'transfers' collection
    transfers_collection.insert_one(transfer, session=session)

    print("Transaction successful")

    return

# Below function is used to send inputs to the callback function so that inputs are not hard coded in callback() and it can be reused.
def callback_wrapper(s):
    callback(
        s,
        transfer_id="TR218721873", # these are inputs
        account_id_receiver="MDB343652528",
        account_id_sender="MDB574189300",
        transfer_amount=100,
    )


# Step 2: Start a client session. A with statement (context manager in Python) is used and start_session() function iss called on client
with client.start_session() as session:
    # Step 3: Use with_transaction to start a transaction, execute the callback, and commit (or cancel on error). 
    # The with_trnsaction() method of session object is called.
    session.with_transaction(callback_wrapper)


client.close()